This Notebook is used for testing code. Content can be ignored for now

In [ ]:
import sys
import os
# Add the parent directory of the cubats module to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

OPENSLIDE_PATH = r"C:\\Users\\mlser\\openslide-bin-4.0.0.6-windows-x64\\openslide-bin-4.0.0.6-windows-x64\\bin"
if hasattr(os, "add_dll_directory"):
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else: 
    import openslide
import cubats.registration as registration
from cubats.slide_collection.slide_collection import SlideCollection
from cubats import segmentation
import cubats.segmentation as segmentation
from cubats import cutils as cutils
from cubats.slide_collection import tile_processing
from  PIL import Image
import shutil
import valis


In [ ]:
import logging

# Set up logging
logging.basicConfig(filename='registration.log', level=logging.INFO, format='%(asctime)s %(message)s')

# Define the source and destination directories
patient_data_dir = r"E:\patient_data"
valis_out_dir = r"E:\valis_out"

# Define the required space in bytes (e.g., 100 GB)
required_space_per_folder = 80 * 1024 * 1024 * 1024  # 100 GB
total_available_space = 590 * 1024 * 1024 * 1024  # 530 GB

# Calculate the maximum number of folders that can be processed
max_folders = total_available_space // required_space_per_folder
print(f"Maximum number of folders that can be processed: {max_folders}")

def has_enough_space(path, required_space):
    """Check if there is enough space available at the given path."""
    total, used, free = shutil.disk_usage(path)
    logging.info(f"Checking space: Total={total}, Used={used}, Free={free}")
    return free >= required_space

# Get the list of folders in patient_data
patient_folders = [f for f in os.listdir(patient_data_dir) if os.path.isdir(os.path.join(patient_data_dir, f))]

# Initialize the counter for processed folders
processed_folders = 0

# Iterate over each folder
for folder_name in patient_folders:
    if processed_folders >= max_folders:
        break

    slide_src_dir = os.path.join(patient_data_dir, folder_name)
    results_dst_dir = os.path.join(valis_out_dir, folder_name, "valis")
    registered_slide_dest_dir = os.path.join(results_dst_dir, "registered_slides")

    # Check if the folder name is not in valis_out
    if not os.path.exists(results_dst_dir):
        # Create the destination directory and the "valis" folder inside it
        os.makedirs(results_dst_dir, exist_ok=True)

        # Check if there is enough space available
        if has_enough_space(valis_out_dir, required_space_per_folder):
            try:
                # Perform the registration
                logging.info(f"Starting registration for {folder_name}")
                registration.register(slide_src_dir, slide_src_dir, microregistration=True, max_non_rigid_registration_dim_px=2000
                                    )
                logging.info(f"Completed registration for {folder_name}")
                processed_folders += 1
            except Exception as e:
                logging.error(f"Error processing {folder_name}: {e}")
                print(f"Error processing {folder_name}. Check the log for details.")
        else:
            logging.warning(f"Not enough space available for {folder_name}. Skipping registration.")
            print(f"Not enough space available for {folder_name}. Skipping registration.")
    else:
        logging.info(f"Results already exist for {folder_name}. Skipping registration.")
        print(f"Results already exist for {folder_name}. Skipping registration.")

print(f"Processed {processed_folders} folders.")

In [ ]:
slide_src_dir = r"E:\patient_data\N_2016_000317/"
results_dst_dir = r"E:\valis_out\N_2016_000317\valis"
registered_slide_dest_dir = r"E:\valis_out\N_2016_000317\valis\registered_slides"
import valis

In [ ]:
registrar = valis.registration.Valis(slide_src_dir, results_dst_dir)
rigid_arr, non_rigid, errodf = registrar.register()

In [ ]:
registrar.register_micro(max_non_rigid_registration_dim_px=2000)

In [ ]:
import pyvips
registrar = valis.registration.load_registrar(r"E:\valis_out\N_2016_000317\valis\data\_registrar.pickle")
pyvips.leak_set(True)
registrar.warp_and_save_slides(registered_slide_dest_dir, crop ="overlap")
# for name, slide_obj in registrar.slide_dict.items():
#     dsf = os.path.join(registered_slide_dest_dir,f"{name}.ome.tiff")
#     slide_obj.warp_and_save_slide(dst_f = dsf, crop = "overlap", compression= "WEBP")

In [ ]:
valis.registration.kill_jvm()

In [ ]:
registration.register(slide_src_dir, results_dst_dir, microregistration= True, max_non_rigid_registration_dim_px=2000)

In [ ]:
registration.high_resolution_alignement(slide_src_dir, slide_src_dir, 1.0)

In [ ]:
registration.register(
    slide_src_dir,
    results_dst_dir,
    microregistration=True,
    max_non_rigid_registration_dim_px=2000
)

In [ ]:
path = r"E:\valis_out\N_2016_000167\registered_slides\N_2016_000167_HE.ome.tiff"
n_path = r"C:\Users\mlser\Desktop\ml_model\seg_mod_256_2023-02-15.onnx"
#path = '/Volumes/EXTERNAL_USB/valis out/N_2014_000862/registered_slides/N_2014_000862_HE.ome.tiff'
#n_path = '/Users/moritz.lokal/Downloads/new_model_w_normalizer/seg_monai_1024_2024-10-17.onnx'
tile_size = (512, 512)
segmentation.run_tumor_segmentation(path, n_path, tile_size, output_path=None, normalization=False, inversion=False, plot_results=True)

In [ ]:
src_dir= r"G:\CuBATS_valis\N_2018_000617\registered_slides"
dst_dir = r"F:\CuBATS_out\N_2018_000617"
#segmentation.run_tumor_segmentation(he_slide, model, (512, 512), output_path=None, normalization=False, inversion=False, plot_results=False)
slide_collection = SlideCollection("N_2018_000617", src_dir, dst_dir)


In [ ]:
slide_collection.slides[6].name

In [ ]:
slide_collection.quantify_single_slide(slide_collection.slides[6].name)

In [ ]:
slide_collection.get_dual_antigen_combinations()
slide_collection.get_triplet_antigen_combinations()

In [ ]:
slide_collection.quantification_results

In [ ]:
slide_collection.dual_antigen_results

In [ ]:
test  = slide_collection.mask.tiles.get_tile(17,(50,60)) 

In [ ]:
import numpy as np

m = np.array(test)
pos = np.sum(m == 0)
neg = np.sum(m == 255)
print(f"pos: {pos}")
print(f"neg: {neg}")
print(f"Pos/Neg: {pos/neg}")


In [ ]:
he_slide = r"E:\valis_out\N_2016_000482\valis\N_2016_000482\registered_slides\N_2016_000482_HE.ome.tiff"
model = r"C:\Users\mlser\Desktop\ml_model\seg_mod_256_2023-02-15.onnx"
src_dir= r"E:\valis_out\N_2016_000482\valis\N_2016_000482\registered_slides"
dst_dir = r"E:\valis_out\N_2016_000482"
segmentation.run_tumor_segmentation(he_slide, model, (512, 512), output_path=None, normalization=False, inversion=False, plot_results=False)
slide_collection = SlideCollection("N_2016_000482", src_dir, dst_dir)
slide_collection.quantify_all_slides()
#slide_collection.quantification_results
slide_collection.get_dual_antigen_combinations()
slide_collection.get_triplet_antigen_combinations()

In [ ]:
he_slide = r"E:\valis_out\N_2016_000686\valis\N_2016_000686\registered_slides\N_2016_000686_HE.ome.tiff"
model = r"C:\Users\mlser\Desktop\ml_model\seg_mod_256_2023-02-15.onnx"
src_dir= r"E:\valis_out\N_2016_000686\valis\N_2016_000686\registered_slides"
dst_dir = r"E:\valis_out\N_2016_000686"
segmentation.run_tumor_segmentation(he_slide, model, (512, 512), output_path=None, normalization=False, inversion=False, plot_results=False)
slide_collection = SlideCollection("N_2016_000686", src_dir, dst_dir)
slide_collection.quantify_all_slides()
#slide_collection.quantification_results
slide_collection.get_dual_antigen_combinations()
slide_collection.get_triplet_antigen_combinations()

In [ ]:
he_slide = r"E:\valis_out\N_2016_000701\valis\N_2016_000701\registered_slides\N_2016_000701_HE.ome.tiff"
model = r"C:\Users\mlser\Desktop\ml_model\seg_mod_256_2023-02-15.onnx"
src_dir= r"E:\valis_out\N_2016_000701\valis\N_2016_000701\registered_slides"
dst_dir = r"E:\valis_out\N_2016_000701"
#segmentation.run_tumor_segmentation(he_slide, model, (512, 512), output_path=None, normalization=False, inversion=False, plot_results=False)
slide_collection = SlideCollection("N_2016_000701", src_dir, dst_dir)
slide_collection.quantify_all_slides()
#slide_collection.quantification_results
slide_collection.get_dual_antigen_combinations()
slide_collection.get_triplet_antigen_combinations()

In [ ]:
src_dir= r"G:\organoid_out\N23_861\registered_slides"
dst_dir = r"G:\organoid_out\N23_861\cubats"
slide_collection = SlideCollection("N23_861", src_dir, dst_dir)
slide_collection.quantify_all_slides()
slide_collection.quantification_results

In [ ]:
slide_collection.get_dual_antigen_combinations()
slide_collection.get_triplet_antigen_combinations()

In [ ]:
src_dir= r"E:\valis_out\N_2015_001043\valis\registered_slides"
dst_dir = r"E:\valis_out\N_2015_001043"
slide_collection = SlideCollection("N_2015_001043", src_dir, dst_dir)
slide_collection.quantify_all_slides()
slide_collection.quantification_results

In [ ]:
src_dir= r"E:\valis_out\N_2016_000008\valis\registered_slides"
dst_dir = r"E:\valis_out\N_2016_000008"
slide_collection = SlideCollection("N_2016_000008", src_dir, dst_dir)
slide_collection.quantify_all_slides()
slide_collection.quantification_results

In [ ]:
src_dir= r"E:\valis_out\N_2016_000021\valis\registered_slides"
dst_dir = r"E:\valis_out\N_2016_000021"
slide_collection = SlideCollection("N_2016_000021", src_dir, dst_dir)
slide_collection.quantify_all_slides()
slide_collection.get_dual_antigen_combinations()
slide_collection.get_triplet_antigen_combinations()
slide_collection.quantification_results

In [ ]:
src_dir= r"E:\valis_out\N_2016_000167\valis\registered_slides"
dst_dir = r"E:\valis_out\N_2016_000167"
slide_collection = SlideCollection("N_2016_000167", src_dir, dst_dir)
slide_collection.quantify_all_slides()
slide_collection.get_dual_antigen_combinations()
slide_collection.get_triplet_antigen_combinations()
slide_collection.quantification_results

In [ ]:
# slide_collection.dual_antigen_results


In [ ]:
slide = slide_collection.slides[5]
slide.properties

In [ ]:
slide_name = "N_2014_000862_ROR2"
slide = [slide for slide in slide_collection.slides if slide.name == slide_name][0]
print(slide.name)

In [ ]:
slide_collection.quantification_results.loc[len(slide_collection.quantification_results)] = slide.quantification_summary

In [ ]:
slide.quantification_summary

In [ ]:
slide_collection.quantification_results

In [ ]:
slide_collection.quantification_results.to_csv(slide_collection.data_dir + '/quantification_results.csv', sep=",",index=False,encoding='utf-8')


In [ ]:
import pickle
pickle.dump(slide.quantification_summary, open(slide_collection.pickle_dir + "/quantification_summary.pickle", "wb"))

In [ ]:
import os
os.path.join(slide_collection.tiles_dir,slide.name,"dab")

In [ ]:
slide_collection.slides[1].summarize_quantification_results()

In [ ]:
slide.quantification_summary['Background (%)']

In [ ]:
slide_collection.quantification_results.loc[len(slide_collection.quantification_results)] = slide.quantification_summary

In [ ]:
slide_collection.quantification_results